In [ ]:
import tensorflow as tf
import os



In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('CPU')

In [ ]:
gpus

In [ ]:
# avoid OOM errors by setting GPU memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


Remove dodgy images

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
data_dir = 'Data'

In [ ]:

os.listdir(os.path.join(data_dir, 'happy')) # gets the image or file on os and file folder

In [ ]:
image_exts = ['jpeg','jpg','bmp','png']

In [ ]:
image_exts

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)): # this lget all the image in certain file
        print(image)

In [ ]:
img = cv2.imread(os.path.join('Data','happy','05-12-21-happy-people.jpg')) # this ready the image to numpy array

In [ ]:
img.shape # this show us the lenght hieght and width also if its rgb or not

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # this output the image also put a rgb color instead of weird blue skin

this code remove not valid image from our os like if it throws of some particular error it will remove it

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)): 
        image_path = os.path.join(data_dir, image_class,image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
    

Load Data

In [ ]:
tf.data.Dataset??

In [ ]:
import numpy as np


In [ ]:
# tf.keras.utils.image_dataset_from_directory('Data',batch_size=8,image_size=(128,128))| # this is the example of the code below of this also let you customize image size and batch size 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('Data') # this creates an automatic label for us like batch and others 

we can say its a generator/ iterator where you can loop back to back
mostly use from huge data

In [ ]:
data_iterator = data.as_numpy_iterator() # converting data to numpy_iterator | this allowing us to loop to its|

In [ ]:
data_iterator # need to run

In [ ]:
# get another batch from the iterator every time you run this it change
batch = data_iterator.next() #| this allowing us to grab it the loop on top|

In [ ]:
batch[0].shape # 2 here in len(batch) is images and labels # using shape gets the batch of 32 i mean show 

In [ ]:
# same as this every time your run it it change the binary
batch[1] # our label this show the label sad and happy 0 is happy 1 is sad

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20)) # plt subplots plot the image 
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

2. Preprocess data

In [ ]:
scaled = batch[0] / 255

In [ ]:
scaled.max()

In [ ]:
data = data.map(lambda x,y: (x/255,y)) # map lallows us to apply a particular transformation as our data preprocessing in pipelines 
# x is our data and y is taaarget so x/255,y

In [ ]:
scaled_iterator = data.as_numpy_iterator() # the as numpy let us access to our iterator and next will be the next batch

In [ ]:
batch = scaled_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20)) # plt subplots plot the image 
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img) # dont use the astype(int) because we're deviding our x to 255 and astype is converting to 0,1 means thers no image in it
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
batch[0].min()

Split Data

In [ ]:
train_size = int(len(data)*.7) # training size 70% of data 
val_size = int(len(data)*.2)+1 # 20% assign to validation  also this is 1
test_size = int(len(data)*.1)+1 # and this is 1
# we add +1 to increase it and the total equal is 7 means we got the len(data)

Skip/Take Method


In [ ]:
train = data.take(train_size) # here we're saying how many batches we  want to allocate on our training data reminder your data should be shuffeld train is equal to 4 batches and 2 will be pass on train size
val = data.skip(train_size).take(val_size) # first is we gonna skip our batches that we already aloccated on our training partition and we're gonna take our last to our val data val_size
test = data.skip(train_size+val_size).take(test_size)# this skill train data and val dataa and take test data

Deep Model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:

model = Sequential()

In [ ]:
# watch this vid if you forgot what are you what are you writing here https://youtu.be/jztwpsIzEGc?list=LL&t=3103
model.add(Conv2D(16,(3,3),1, activation = 'relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy']) # we use loss=tf.losses.BinaryCrossentropy() because we're passing 1, 0 and we're getting the metrics of accuracy and using adam optimizer but theres more

In [ ]:
model.summary() # watch this for full explanation how it works https://youtu.be/jztwpsIzEGc?list=LL&t=3577

3.2 Train

In [ ]:
logdir='log' 


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) # if you want to save your model or some checkpoint u need this or specific login (callback)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color= 'blue', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='Val_loss')
plt.suptitle('loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color= 'blue', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='Val_loss')
plt.suptitle('loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

Evaluate Performance

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy # usually use for clasification problem

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
len(test)

In [ ]:
for batch in test.as_numpy_iterator():
    x,y = batch
    yhat = model.predict(x)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'precission:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Retrain

Training our model that it never seen before

In [ ]:
img = cv2.imread('sad_test.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resized_image, 0 ).shape

In [ ]:
yhat = model.predict(np.expand_dims(resize/255 ,0)) # our neural network expect us to pas batch of image so what we need to do is this code calculate it in different parenthesis or array put it inside of the list3

In [ ]:
yhat # it successfully predicted that this is happy because of [0.42910603] in first we have 0

In [ ]:
if yhat > 0.5:
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is happy')

Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models', 'CNN_image.h5')) # we use h5 model means we're doing serialization so we can store it like a disk same as rar dataset

In [ ]:
new_model = load_model(os.path.join('models', 'CNN_image.h5'))

In [ ]:
new_model